In [1]:
import pandas as pd
import ast

In [2]:
llama = pd.read_csv('selected_clusters/llama_90b_top5_step2_kmeans_cosine.csv')

llama['image'] = llama['image'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
llama.head()

,verbs,cluster,image,target,dataset,final_cluster
0,adjusting,0,"[inflating_186.jpg, inflating_197.jpg, inflati...","['inflating', 'inflating', 'inflating', 'infla...","['placeholder', 'placeholder', 'placeholder', ...",81
1,adjusting,1,"[yanking_43.jpg, putting_356.jpg, putting_54.j...","['yanking', 'putting', 'putting', 'putting', '...","['placeholder', 'placeholder', 'placeholder', ...",476
2,admiring,0,"[putting_331.jpg, walking_303.jpg, applying_27...","['putting', 'walking', 'applying', 'admiring',...","['placeholder', 'placeholder', 'placeholder', ...",325
3,admiring,1,"[congregating_271.jpg, hanging_73.jpg, shoppin...","['congregating', 'hanging', 'shopping', 'paint...","['placeholder', 'placeholder', 'placeholder', ...",325
4,admiring,2,"[skiing_8.jpg, cresting_206.jpg, admiring_176....","['skiing', 'cresting', 'admiring', 'admiring',...","['placeholder', 'placeholder', 'placeholder', ...",325


In [3]:
clusters = llama.groupby('final_cluster')[['verbs']].agg(list).reset_index()

clusters

,final_cluster,verbs
0,0,"[scoring, scoring, scoring]"
1,1,"[tattooing, tattooing, tattooing, tattooing]"
2,2,"[tying, tying, tying, tying, tying, tying, tying]"
3,3,"[poking, pressing, pulling, pushing, turning]"
4,4,"[hoisting, hoisting, hoisting, lifting]"
...,...,...
599,599,[shooting]
600,600,"[rubbing, rubbing, rubbing, rubbing]"
601,601,"[flaming, flaming]"
602,602,"[waving, waving]"


In [4]:
images = llama.explode('image')[['image', 'final_cluster']]

images = images.merge(clusters, on='final_cluster', how='left')

images = images.groupby('image').agg(list).reset_index()

images['targets'] = images['verbs'].apply(lambda x: list(set([item for sublist in x for item in sublist])))
images

,image,final_cluster,verbs,targets
0,adjusting_0.jpg,"[81, 596]","[[adjusting, installing, tuning], [smiling, sm...","[tuning, smiling, installing, adjusting]"
1,adjusting_1.jpg,[81],"[[adjusting, installing, tuning]]","[tuning, installing, adjusting]"
2,adjusting_10.jpg,"[81, 233, 596, 344]","[[adjusting, installing, tuning], [examining],...","[smiling, tuning, installing, working, examini..."
3,adjusting_100.jpg,[476],"[[adjusting, arranging, buttoning, fixing, fix...","[fixing, arranging, buttoning, adjusting]"
4,adjusting_101.jpg,"[476, 217, 17, 259]","[[adjusting, arranging, buttoning, fixing, fix...","[glaring, smiling, talking, sitting, fixing, a..."
...,...,...,...,...
126056,yawning_95.jpg,"[515, 32]","[[covering], [yawning, yawning]]","[covering, yawning]"
126057,yawning_96.jpg,"[550, 32]","[[sleeping, sleeping], [yawning, yawning]]","[yawning, sleeping]"
126058,yawning_97.jpg,"[239, 32]","[[resting, resting], [yawning, yawning]]","[yawning, resting]"
126059,yawning_98.jpg,"[550, 32]","[[sleeping, sleeping], [yawning, yawning]]","[yawning, sleeping]"


In [5]:
import json
import os

# Ensure the results directory exists
os.makedirs('results', exist_ok=True)

# Create dictionary mapping image to targets
image_targets = dict(zip(images['image'], images['targets']))

# Save to JSON file
with open('results/results_llama_90b_top5_kmeans.json', 'w') as f:
    json.dump(image_targets, f)

In [6]:
with open('results/results_llama_90b_top5_kmeans.json', 'r') as f_read:
    loaded_image_targets = json.load(f_read)

# Display the number of images and a sample
print(f"Loaded {len(loaded_image_targets)} images.")
print(list(loaded_image_targets.items())[:3])

Loaded 126061 images.
[('adjusting_0.jpg', ['tuning', 'smiling', 'installing', 'adjusting']), ('adjusting_1.jpg', ['tuning', 'installing', 'adjusting']), ('adjusting_10.jpg', ['smiling', 'tuning', 'installing', 'working', 'examining', 'adjusting'])]
